In [ ]:
import pickle
import pandas as pd
from pathlib import Path
import gzip
from plotnine import *
from socialgene.clustermap.serialize import SerializeToClustermap
from socialgene.neo4j.neo4j import GraphDriver # grab the the neo4j connection
from socialgene.config import env_vars
env_vars["NEO4J_URI"] = "bolt://localhost:7687"

pickle_path ='BGC0000946.pickle.gz'
json_path = 'clinker.json'

In [ ]:
# The pickle file is a serialized object that contains the search results from the SocialGene search engine results 
# from searching all of RefSeq for similar genetic regions to the BGC0000946 cluster.

with gzip.open(pickle_path, 'rb') as f:
    search_object = pickle.load(f)

Making trees of each vibrioferrin BGC protein

In [ ]:
filenames = set()
for i in search_object.link_df.iterrows():
    filename = f"{i[1].query_feature.external_id}_{i[1].query_feature.description.replace(' ', '_')}.fasta"
    filenames.add(filename)
    with open(Path("fasta_for_tree", filename), "a") as h:
        single_fasta_entry=f">{i[1].target_feature.parent.external_id}_{i[1].target_feature.external_id}_{i[1].target_feature.start}\n{i[1].target_feature.protein.sequence}\n"
        h.write(single_fasta_entry)       
        

In [ ]:
import subprocess

for filename in filenames:
    with open(Path("align_for_tree", filename.replace(".fasta", ".mafft_aln")), "a") as h:
        h.write(subprocess.run(["mafft",  "--thread", "22", "--auto", Path("fasta_for_tree", filename)], capture_output=True).stdout.decode())


In [ ]:
import subprocess

for filename in filenames:
    with open(Path("trees", filename.replace(".fasta", ".newick")), "a") as h:
        h.write(subprocess.run(["FastTree",  Path("align_for_tree", filename.replace(".fasta", ".mafft_aln"))], capture_output=True).stdout.decode())


In [ ]:
names= []
for i in search_object.link_df.iterrows():
        names.append(f"{i[1].target_feature.parent.external_id}_{i[1].target_feature.external_id}_{i[1].target_feature.start}")

In [ ]:
for filename in filenames:
    with open(Path("trees", filename.replace(".fasta", ".newick")), "r") as h:
        tree=h.read()
        for i in names:
            if "_".join(i.split('_')[0:2]) in df.target_nucleotide_sequence.to_list():
                tree = tree.replace(i, df.loc[df.target_nucleotide_sequence == "_".join(i.split('_')[0:2]), "genus"].to_list()[0])
        with open(Path("trees", f'genus_{filename.replace(".fasta", ".newick")}'), "w") as h2:
            h2.write(tree)
            